In [1]:
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objects as go
import numpy as np
import sys
from pathlib import Path

# Adiciona o caminho src ao sys.path
sys.path.insert(0, str(Path().resolve().parent))

import utils as utils
import setups.emas as emas
import setups.stopgain as StopGain
import setups.stoploss as StopLoss


In [14]:
def plot_trades(data, trades, start_date, short_period, long_period):
    fig = go.Figure()

    data = data[data['open_time'] >= start_date]  # Filtra os dados a partir do start_date

    fig.add_trace(go.Candlestick(
        x=data['open_time'],
        open=data['open'],
        high=data['high'],
        low=data['low'],
        close=data['close'],
        name='Candlesticks',
        increasing_line_color='rgba(144, 238, 144, 0.7)',  # Verde um pouco mais claro
        decreasing_line_color='rgba(255, 99, 71, 0.7)',  # Vermelho um pouco mais claro
        increasing_fillcolor='rgba(144, 238, 144, 0.5)',
        decreasing_fillcolor='rgba(255, 99, 71, 0.5)',
    ))

    fig.add_trace(go.Scatter(
        x=data['open_time'],
        y=data[f'ema_{short_period}'],
        mode='lines',
        name=f'ema {short_period}',
        line=dict(color='yellow', width=1)
    ))
    fig.add_trace(go.Scatter(
        x=data['open_time'],
        y=data[f'ema_{long_period}'],
        mode='lines',
        name=f'ema {long_period}',
        line=dict(color='rgb(148,0,211)', width=1)
    ))

    for trade in trades:
        if trade['open_time'] >= start_date:  # Plota apenas trades a partir do start_date
            fig.add_trace(go.Scatter(
                x=[trade['open_time']],
                y=[trade['buy_price']],
                hovertext=[{
                    'Preço de Compra': f"{trade['buy_price']:.2f}", 
                    'Stoploss': f"{trade['stoploss']:.2f}", 
                    'Stopgain': f"{trade['stopgain']:.2f}"
                }],
                mode='markers',
                marker=dict(color='rgb(100, 149, 237)', size=15, symbol='circle'),  # Azul mais forte
                name='Buy'
            ))

            if trade['result'] == 'StopLoss':
                color = 'rgb(255, 69, 0)'  # Vermelho mais forte
                symbol = 'triangle-down'
                result_text = f"-{trade['outcome']:.2f}%"
            elif trade['result'] == 'StopGain':
                color = 'rgb(60, 179, 113)'  # Verde mais forte
                symbol = 'triangle-up'
                result_text = f"+{trade['outcome']:.2f}%"
            
            fig.add_trace(go.Scatter(
                x=[trade['close']],
                y=[trade['close_price']],
                hovertext=[{
                    'Fechou em': f"{trade['close_price']:.2f}", 
                    'Preço de Compra': f"{trade['buy_price']:.2f}", 
                    'Resultado': result_text,
                    'Saldo': f"{trade['saldo']:.2f}"
                }],
                mode='markers',
                marker=dict(color=color, size=15, symbol=symbol),
                name=trade['result']
            ))

    fig.update_layout(
        title='Trades',
        xaxis_title='Time',
        yaxis_title='Price',
        template='plotly_dark'
    )

    fig.update_yaxes(
        fixedrange=False,
        autorange=True 
    )

    fig.show()

def adjust_date(start_date, timeframe, num_extra_candles=150):
    """
    Ajusta a data de início para adicionar candles extras, garantindo que as EMAs sejam calculadas corretamente.
    
    Parameters:
    - start_date: data e hora em que as transações devem começar
    - timeframe: o intervalo de tempo entre as velas (ex: '15m', '5m', etc.)
    - num_extra_candles: número de velas extras a serem adicionadas para calcular as EMAs (default: 150)
    
    Returns:
    - adjusted_start_date: data ajustada para incluir velas extras para o cálculo das EMAs
    """
    start_datetime = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
    
    # Converter timeframe para minutos
    timeframe_minutes = int(timeframe[:-1])  # Ex: '15m' vira 15, '5m' vira 5
    total_minutes_to_subtract = timeframe_minutes * num_extra_candles  # Ex: 15 minutos * 150 velas
    
    # Ajustar a data inicial subtraindo o total de minutos necessários
    adjusted_start_datetime = start_datetime - timedelta(minutes=total_minutes_to_subtract)
    
    return adjusted_start_datetime.strftime('%Y-%m-%d %H:%M:%S')

def calculate_sharpe_ratio(returns, risk_free_rate=0.05):
    excess_returns = returns - risk_free_rate
    mean_excess_return = np.mean(excess_returns)
    std_excess_return = np.std(excess_returns)
    sharpe_ratio = mean_excess_return / std_excess_return if std_excess_return != 0 else 0
    return sharpe_ratio

# Função para carregar e padronizar dados dos arquivos CSV
def load_and_prepare_data(file_15m, file_5m):
    # Carregar os dados dos arquivos CSV
    df_15m = pd.read_csv(file_15m)
    df_5m = pd.read_csv(file_5m)

    # Padronizar os nomes das colunas para minúsculas e substituir espaços por underscores
    df_15m.columns = [col.lower().replace(' ', '_') for col in df_15m.columns]
    df_5m.columns = [col.lower().replace(' ', '_') for col in df_5m.columns]

    return df_15m, df_5m

In [15]:
# Parâmetros
start_date = "2023-08-11 00:00:00"
end_date = "2024-08-01 00:00:00"
timeframe = '15m'
alavancagem = 1
short_period = 5
long_period = 15
stop_candles = 17
ratio = 4.1
taxa_por_operacao = 0.03
ativo = 'BTCUSDT'

adjusted_start_date = adjust_date(start_date, timeframe, long_period)


In [16]:
# Carregar os dados
df_15m, df_5m = load_and_prepare_data('BTC_15m_candles.csv', 'BTC_5m_candles.csv')

# Definir o tempo inicial e o tempo final
start_datetime = pd.to_datetime(adjusted_start_date)
end_datetime = pd.to_datetime(end_date)

# Quantidade de candles necessários antes do start_date para o cálculo da EMA mais longa (no caso, EMA de 15 períodos)
num_candles_for_ema = long_period  # Se long_period = 15, precisaremos de 15 candles

# Filtrar os dados com base no tempo anterior ao start_date para calcular as EMAs corretamente
df_15m['open_time'] = pd.to_datetime(df_15m['open_time'])
df_5m['open_time'] = pd.to_datetime(df_5m['open_time'])

# Filtrando velas 15m para incluir 15 candles anteriores ao start_date
df_15m_for_ema = df_15m[df_15m['open_time'] < start_datetime].tail(num_candles_for_ema)  # Pegando os candles anteriores
df_15m = df_15m[(df_15m['open_time'] >= start_datetime) & (df_15m['open_time'] <= end_datetime)]  # Candles dentro do período de operação
df_15m = pd.concat([df_15m_for_ema, df_15m])

# Filtrando velas 5m para incluir 15 candles anteriores ao start_date
df_5m_for_ema = df_5m[df_5m['open_time'] < start_datetime].tail(num_candles_for_ema)
df_5m = df_5m[(df_5m['open_time'] >= start_datetime) & (df_5m['open_time'] <= end_datetime)]
df_5m = pd.concat([df_5m_for_ema, df_5m])

# Selecionar o dataframe correto com base no timeframe
if timeframe == '15m':
    data = df_15m.copy()
elif timeframe == '5m':
    data = df_5m.copy()

# Verificar se temos dados para o período
if data.empty:
    print("No data available for the given period.")
else:
    # Calcular as EMAs desde o começo dos dados carregados (incluindo os candles anteriores ao start_date)
    data['close'] = data['close'].astype(float)
    data['low'] = data['low'].astype(float)
    data['high'] = data['high'].astype(float)
    data[f'ema_{short_period}'] = data['close'].ewm(span=short_period, adjust=False).mean()
    data[f'ema_{long_period}'] = data['close'].ewm(span=long_period, adjust=False).mean()
    data['ema_80'] = data['close'].ewm(span=80, adjust=False).mean()

    # Agora, filtramos de novo para que as operações só comecem a partir do start_date
    data = data[data['open_time'] >= start_datetime]

    # A partir daqui, o código continua normalmente, calculando e executando as transações...
    saldo_inicial = 1000
    saldo = saldo_inicial * alavancagem  # Ajustando o saldo para considerar a alavancagem

    max_saldo = saldo
    min_saldo_since_max = saldo
    min_saldo_since_start = saldo
    max_drawdown = 0
    initial_drawdown = 0
    perdas = []
    ganhos = []

    risk_free_rate = 0.05  # Determinar taxa de juros de títulos públicos para o período testado

    comprado = False

    results = {}
    trades = []

    # Loop principal para processar operações
    for i in range(999, len(data)):  # Começa a partir do 999 para garantir que EMAs já estejam calculadas
        year = data['open_time'].iloc[i - 1].year  # Usando a coluna 'open_time' que é do tipo datetime
        month = data['open_time'].iloc[i - 1].month

        if year not in results:
            results[year] = {}
        if month not in results[year]:
            results[year][month] = {
                'open_trades': 0,
                'lucro': 0,
                'successful_trades': 0,
                'failed_trades': 0,
                'perda_percentual_total': 0,
                'saldo_inicial': saldo,
                'saldo_final': saldo,
                'max_drawdown': 0
            }

        if comprado:
            if StopLoss.sell_stoploss(data['low'].iloc[i - 1], stoploss):
                loss_percentage = utils.calculate_loss_percentage(buy_price, stoploss)
                results[year][month]['failed_trades'] += 1
                results[year][month]['perda_percentual_total'] += loss_percentage + taxa_por_operacao
                saldo -= saldo * ((loss_percentage + taxa_por_operacao) / 100)
                results[year][month]['saldo_final'] = saldo
                comprado = False
                
                print(f"{data['open_time'].iloc[i - 1]} - VENDEMOS a {round(stoploss, 2)} com PREJUÍZO de {round(loss_percentage, 2)}% indo para {round(saldo, 2)} de saldo")

                trade['close_price'] = stoploss
                trade['close'] = data['open_time'].iloc[i - 1]
                trade['outcome'] = loss_percentage
                trade['result'] = 'StopLoss'
                trade['saldo'] = saldo
                trades.append(trade)
                perdas.append(-(loss_percentage + taxa_por_operacao))

                if saldo < min_saldo_since_max:
                    min_saldo_since_max = saldo
                    drawdown = (max_saldo - min_saldo_since_max) / max_saldo * 100
                    
                    if drawdown > max_drawdown:
                        max_drawdown = drawdown
                        
                    results[year][month]['max_drawdown'] = max_drawdown

                if saldo < min_saldo_since_start:
                    min_saldo_since_start = saldo
                    investment_drawdown = (saldo_inicial - min_saldo_since_start) / saldo_inicial * 100
                    if investment_drawdown > initial_drawdown:
                        initial_drawdown = investment_drawdown

                continue
                
            elif StopGain.sell_stopgain(data['high'].iloc[i - 1], stopgain):
                profit = utils.calculate_gain_percentage(buy_price, stopgain)
                results[year][month]['lucro'] += profit - taxa_por_operacao
                results[year][month]['successful_trades'] += 1
                saldo += saldo * ((profit - taxa_por_operacao) / 100)
                results[year][month]['saldo_final'] = saldo
                comprado = False

                print(f"{data['open_time'].iloc[i - 1]} - VENDEMOS a {round(stopgain, 2)} com LUCRO de {round(profit, 2)}% indo para {round(saldo, 2)} de saldo")

                trade['close_price'] = stopgain
                trade['close'] = data['open_time'].iloc[i - 1]
                trade['outcome'] = profit
                trade['result'] = 'StopGain'
                trade['saldo'] = saldo
                trades.append(trade)

                ganhos.append(profit - taxa_por_operacao)

                if saldo > max_saldo:
                    max_saldo = saldo
                    min_saldo_since_max = saldo

                continue

        if not comprado:
            if emas.buy_double_ema_breakout(data.iloc[i - 5:i], f'ema_{short_period}', f'ema_{long_period}'):
                results[year][month]['open_trades'] += 1
                buy_price = data['high'].iloc[i - 2]
                stoploss = StopLoss.set_sell_stoploss_min_candles(data.iloc[i - (stop_candles + 1):i], stop_candles)  # Usando os parâmetros fornecidos
                if taxa_por_operacao != 0:
                    saldo -= saldo * taxa_por_operacao / 100
                results[year][month]['saldo_final'] = saldo
                stopgain = StopGain.set_sell_stopgain_ratio(buy_price, stoploss, ratio)
                comprado = True

                loss_percentage = utils.calculate_loss_percentage(buy_price, stoploss)
                gain_percentage = utils.calculate_gain_percentage(buy_price, stopgain)

                print(f"{data['open_time'].iloc[i - 1]} - COMPRAMOS a {round(buy_price, 2)} com stoploss em {round(stoploss, 2)} ({round(loss_percentage, 2)}% de perda) e stopgain em {round(stopgain, 2)} ({round(gain_percentage, 2)}% de ganho)")

                trade = {
                    'open_time': data['open_time'].iloc[i - 1],
                    'buy_price': buy_price,
                    'stoploss': stoploss,
                    'stopgain': stopgain,
                    'close_price': 0,
                    'close': 0,
                    'outcome': 0,
                    'result': '',
                    'saldo': saldo
                }
                trades.append(trade)
                continue

    descricao_setup = f"ema {short_period}/{long_period} rompimento, stopgain ratio {ratio} e stoploss {stop_candles} candles"

    overall_sharpe_ratio = calculate_sharpe_ratio(np.array(ganhos + perdas), 0.15)

# Verificação e exibição dos resultados
if results:
    # Verificar se os anos e meses dentro de 'results' têm valores válidos
    first_year = list(results.keys())[0]
    first_month = list(results[first_year].keys())[0]
    
    last_year = list(results.keys())[-1]
    last_month = list(results[last_year].keys())[-1]

    # Verificando se há valores de saldo inicial e final no primeiro e último mês do período
    saldo_inicial = results[first_year][first_month]['saldo_inicial']
    saldo_final = results[last_year][last_month]['saldo_final']

    print(f"Saldo inicial: {saldo_inicial:.2f}")
    print(f"Saldo final: {saldo_final:.2f}")

    # Verificar se o saldo final é maior ou menor que o inicial
    if saldo_inicial <= saldo_final:
        print(f"Resultado final: {(saldo_final / saldo_inicial - 1) * 100:.2f}%")
    else:
        print(f"Resultado final: {((1 - (saldo_final / saldo_inicial)) * -1) * 100:.2f}%")

    # Exibir drawdowns
    print(f"Drawdown inicial: {initial_drawdown:.2f}%")
    print(f"Drawdown máximo: {max_drawdown:.2f}%")

    # Exibir outros resultados, como número de operações, sharpe ratio, etc.
    print("Total:")
    print(f"Operações realizadas: {sum([results[year][month]['open_trades'] for year in results for month in results[year]])}")
    print(f"Sharpe Ratio: {overall_sharpe_ratio:.2f}")
    try:
        print(f"Taxa de acerto: {sum([results[year][month]['successful_trades'] for year in results for month in results[year]]) / sum([results[year][month]['open_trades'] for year in results for month in results[year]]) * 100:.2f}%")
    except ZeroDivisionError:
        print(f"Taxa de acerto: 0")
    print(f"Trades de sucesso: {sum([results[year][month]['successful_trades'] for year in results for month in results[year]])}")
    print(f"Soma dos ganhos: {sum([results[year][month]['lucro'] for year in results for month in results[year]]):.2f}%")

    try:
        print(f"Ganho médio por trade: {sum([results[year][month]['lucro'] for year in results for month in results[year]]) / sum([results[year][month]['successful_trades'] for year in results for month in results[year]]) :.2f}%")
    except ZeroDivisionError:
        print(f"Ganho médio por trade: 0")

    print(f"Trades em prejuízo: {sum([results[year][month]['failed_trades'] for year in results for month in results[year]])}")
    print(f"Soma das perdas: {sum([results[year][month]['perda_percentual_total'] for year in results for month in results[year]]):.2f}%")
    try:
        print(f"Perda média por trade: {sum([results[year][month]['perda_percentual_total'] for year in results for month in results[year]]) / sum([results[year][month]['failed_trades'] for year in results for month in results[year]]) :.2f}%")
    except ZeroDivisionError:
        print(f"Perda média por trade: 0")
        # Finalização e exibição do resultado final
    if saldo_inicial <= saldo_final:
        print(f"Resultado final: {(saldo_final / saldo_inicial - 1) * 100:.2f}%")
    else:
        print(f"Resultado final: {((1 - (saldo_final / saldo_inicial)) * -1) * 100:.2f}%")

    print(f"Saldo inicial: {saldo_inicial:.2f}")
    print(f"Saldo final: {saldo_final:.2f}")
    print("-------------------")
    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}: Teste finalizado: {ativo} - {timeframe}.")
    print(f"Setup: {descricao_setup}")
else:
    print("Nenhum resultado disponível para o período selecionado.")

2023-08-21 11:45:00 - COMPRAMOS a 26001.69 com stoploss em 25880.2 (0.47% de perda) e stopgain em 26499.8 (1.92% de ganho)
2023-08-21 15:15:00 - VENDEMOS a 25880.2 com PREJUÍZO de 0.47% indo para 994.23 de saldo
2023-08-21 17:15:00 - COMPRAMOS a 26039.82 com stoploss em 25812.0 (0.87% de perda) e stopgain em 26973.88 (3.59% de ganho)
2023-08-22 16:30:00 - VENDEMOS a 25812.0 com PREJUÍZO de 0.87% indo para 984.44 de saldo
2023-08-22 16:45:00 - COMPRAMOS a 26046.14 com stoploss em 25796.31 (0.96% de perda) e stopgain em 27070.44 (3.93% de ganho)
2023-08-22 17:30:00 - VENDEMOS a 25796.31 com PREJUÍZO de 0.96% indo para 973.92 de saldo
2023-08-22 21:15:00 - COMPRAMOS a 25882.54 com stoploss em 25300.0 (2.25% de perda) e stopgain em 28270.95 (9.23% de ganho)
2023-09-11 14:00:00 - VENDEMOS a 25300.0 com PREJUÍZO de 2.25% indo para 950.94 de saldo
2023-09-11 21:15:00 - COMPRAMOS a 25198.63 com stoploss em 24901.0 (1.18% de perda) e stopgain em 26418.91 (4.84% de ganho)
2023-09-12 16:30:00 - V

In [ ]:
#plot_trades(data, trades, pd.to_datetime(start_date), short_period, long_period)